In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import psycopg2
from datetime import datetime
from psycopg2.extras import execute_values
import time

import import_ipynb
import data_structures as ds

importing Jupyter notebook from data_structures.ipynb


In [2]:
headers = {
    "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/79.0.3945.88 Safari/537.36'}

In [3]:
def get_csi_df():
    df = pd.DataFrame()
    cardnames = []
    sets = []
    prices = []
    for p in ['1', '2', '4', '5', '6']:
        page = requests.get('https://www.coolstuffinc.com/buylist_text.php?pa=vbl&gn=mtg&dp=' + p, headers=headers)

        soup = BeautifulSoup(page.content, 'html.parser')

        html = list(soup.children)[0]

        body = list(html.children)[3]

        t = list(body.children)[5]

        segmented = t.get_text().split('\n\n\n\n\n\n')
        for segment in segmented:
            words = segment.split('\n\xa0\n')
            if len(words) == 14:
                words = words[:7]
            if len(words) == 7 and 'Foil' not in words[1] and 'New' not in words[1] and 'Variants' not in words[2]:
                cardname = words[0].replace('\n', '')
                set = words[2]
                price = float(words[5].replace('$', ''))
                cardnames.append(cardname)
                sets.append(set)
                prices.append(price)

        print('CSI page ' + str(p) + ' processed')

    df['CSI_Name'] = pd.Series(cardnames)
    df['CSI_Set'] = pd.Series(sets)
    df['CSI_Price'] = pd.Series(prices)
    df.drop_duplicates(inplace=True)

    return df

In [4]:
def store_csi(df):
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)

    conn = psycopg2.connect(host="localhost", port=5432, database="", user="", password="")
    cur = conn.cursor()

    to_store = pd.DataFrame(columns=['uuid', 'csiDate', 'csiBuyPrice'])
    uuid = []
    prices = []
    for index, row in df.iterrows():
        try:
            uuid.append(ds.get_uuid(cur, row.CSI_Name, row.CSI_Set, 'csi'))
            prices.append(row.CSI_Price)
        except Exception as e:
            pass

    to_store['uuid'] = pd.Series(uuid)
    to_store['csibuyprice'] = pd.Series(prices)
    to_store['csidate'] = datetime.now()
    
    to_store.drop_duplicates(subset=['uuid', 'csidate'], inplace=True)
    
    t_df = tuple(to_store[['uuid', 'csidate', 'csibuyprice']].itertuples(index=False, name=None))
    execute_values(cur, "INSERT INTO csibuydata (uuid, csidate, csibuyprice) VALUES %s", t_df)
    conn.commit()

    print('CSI Stored')
    cur.close()
    conn.close()

In [5]:
def main():
    df = get_csi_df()
    store_csi(df)

In [6]:
if __name__ == '__main__':
    main()

CSI page 1 processed
CSI page 2 processed
CSI page 4 processed
CSI page 5 processed
CSI page 6 processed
CSI Stored
